In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [8]:
url = 'https://listado.mercadolibre.com.pe/carros#D[A:carros]'

encabezados = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
}

response = requests.get(url, headers=encabezados)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

# Verificar si la conexión es exitosa
if response.status_code == 200:
    print("Conexión exitosa con la página")
else:
    print("Error al conectar con la página")

# Extraer nombres de productos (en este caso, títulos de los carros)
prod = soup.find_all('h2', class_='ui-search-item__title')
productos = [i.text for i in prod]

# Verificar si se han encontrado productos
if productos:
    print(f"Productos encontrados: {len(productos)}")
else:
    print("No se encontraron productos")

# Extraer precios
prec = soup.find_all('span', class_='price-tag-fraction')
precios = [i.text for i in prec]

# Verificar si se han encontrado precios
if precios:
    print(f"Precios encontrados: {len(precios)}")
else:
    print("No se encontraron precios")

# Crear DataFrame
df_total = pd.DataFrame({'Nombre Producto': productos, 'Precio': precios})

# Convertir precios a numéricos
df_total['Precio'] = df_total['Precio'].apply(lambda x: re.findall(r'\d+.\d+|\d+', x)[0] if re.findall(r'\d+.\d+|\d+', x) else 0)
df_total['Precio'] = pd.to_numeric(df_total['Precio'])

# Ordenar DataFrame por Precio en orden ascendente
df_total = df_total.sort_values('Precio', ascending=True)

# Imprimir resultados ordenados
print(df_total)

Conexión exitosa con la página
No se encontraron productos
No se encontraron precios
Empty DataFrame
Columns: [Nombre Producto, Precio]
Index: []
